# CPS Project 

To do:
- Simulare su Gazebo un drone dotato di videocamera e imparare ad interpretare i dati prodotti da quest'ultima (con l'aiuto di RVIZ ROS)
- Implementare uno o più aruco codes e utilizzarli come riferimento per lo spostamento del drone

### Gazebo

In [ ]:
# Apertura Gazebo con un drone generico
!cd PX4-Autopilot/
!make px4_sitl gazebo-classic

In [ ]:
# Apertura Gazebo con quadrotore Iris (con videocamera)
!cd PX4-Autopilot/
!make px4_sitl gazebo-classic_iris_opt_flow

!make px4_sitl gazebo-classic_iris_depth_camera

In [ ]:
# Per fare decollare il drone
!commander takeoff # comando da inserire nel terminale di lancio px4_sitl



### Iris model with RGBD camera

https://github.com/ViniciusAbrao/px4_ros2_xrcedds

For now we face some problems, mainly because of a presumed incompatibility between ros2, px4 and gazebo

In [ ]:
# TERMINAL 1
!cd PX4-Autopilot/

# Make PX4 and roslaunch the gazebo simulation:
!make px4_sitl_default gazebo-classic_iris # This after having changed the iris.sdf file in the models folder of PX4
!roslaunch px4 posix_sitl.launch

In [ ]:
# TERMINAL 2
!MicroXRCEAgent udp4 -p 8888

In [ ]:
# TERMINAL 3
!source /opt/ros/foxy/setup.bash
!ros2 run ros1_bridge dynamic_bridge

In [ ]:
# TERMINAL 4
!cd ~/dev/px4_ros2/ros_packages
!source /opt/ros/foxy/setup.bash
!colcon build
!source install/local_setup.bash
!ros2 launch px4_offboard offboard_position_control.launch.py

In [ ]:
# TERMINAL 5
!source /opt/ros/foxy/setup.bash
!ros2 topic list

!source /opt/ros/foxy/setup.bash
!ros2 topic echo /camera/color/image_raw sensor_msgs/msg/Image 

### Micro XRCE 

In [ ]:
# Apertura Micro XRCE Agent (da un diverso terminale)
!MicroXRCEAgent udp4 -p 8888